In [46]:
!apt install swi-prolog

swi-prolog is already the newest version (7.6.4+dfsg-1build1).


0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.
W: Not using locking for read only lock file /var/lib/dpkg/lock-frontend
W: Not using locking for read only lock file /var/lib/dpkg/lock
W: Not using locking for read only lock file /var/cache/apt/archives/lock


In [50]:
KB = """
%  Tell prolog that known/3 will be added later by asserta
:- dynamic known/3. 

% Enter your KB below this line:

% problem(battery) :- \+engine(turning_over), battery(bad).
% battery(bad) :- lights(weak).
% battery(bad) :- radio(weak).
% problem(out_of_gas) :- engine(turning_over), gas_gauge(empty).
% problem(engine_flooded) :- engine(turning_over), smell(gas).

restaurant(the_Visit) :- price(medium), distance(near), accessibility(bicycle), cuisine(western), meal_type(cafe), (vegan_option(vegan) ; vegan_option(non_vegan)), language(english), (opening_hours(morning); opening_hours(afternoon)).
restaurant(maroush) :- price(low), distance(near), accessibility(bicycle), cuisine(middle_eastern), meal_type(snack), (vegan_option(vegan) ; vegan_option(non_vegan)), language(english), (opening_hours(afternoon); opening_hours(evening)).
restaurant(hacibaba) :- price(low), distance(near), accessibility(bicycle), cuisine(middle_eastern), meal_type(snack), (vegan_option(vegan) ; vegan_option(non_vegan)), language(english), (opening_hours(evening); opening_hours(afternoon)).
restaurant(green_Rice) :- price(medium), distance(near), accessibility(bicycle), cuisine(asian), meal_type(restaurant), (vegan_option(vegan) ; vegan_option(non_vegan)), language(english), (opening_hours(morning); opening_hours(afternoon)).
restaurant(tangs) :- price(medium), distance(near), accessibility(bicycle), cuisine(asian), meal_type(restaurant), (vegan_option(vegan) ; vegan_option(non_vegan)), language(english), (opening_hours(afternoon); opening_hours(evening)).
restaurant(zur_Gerichtslaube) :- price(high), distance(far), accessibility(bicycle), cuisine(western), meal_type(restaurant), (vegan_option(vegan) ; vegan_option(non_vegan)), language(english), opening_hours(evening).
restaurant(parker_Bowles) :- price(medium), distance(near), accessibility(bus), cuisine(western), meal_type(restaurant), vegan_option(non_vegan), language(german), (opening_hours(evening)).
restaurant(cocolo_ramen) :- price(medium), distance(far), accessibility(bus), cuisine(asian), meal_type(restaurant), vegan_option(non_vegan), language(english), (opening_hours(afternoon); opening_hours(evening)).


% The code below implements the prompting to ask the user:

price(X) :- menuask(price, X, [low, medium, high]).
distance(X) :- menuask(distance, X, [near, far]).
accessibility(X) :- menuask(accessibility, X, [bicycle, bus]).
cuisine(X) :- menuask(cuisine, X, [asian, middle_eastern, western]).
meal_type(X) :- menuask(meal_type, X, [cafe, snack, restaurant]).
vegan_option(X) :- menuask(vegan_option, X, [vegan, non_vegan]).
language(X) :- menuask(language, X, [english, german]).
opening_hours(X) :- menuask(opening_hours, X, [morning, afternoon, evening]).


% Asking clauses
multivalued(none).

menuask(A, V, MenuList):-
known(yes, A, V), % succeed if true
!.	% stop looking

menuask(A, V, MenuList):-
known(_, A, V), % fail if false
!, fail.

% If not multivalued, and already known, don't ask again for a different value.
menuask(A, V, MenuList):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!.

menuask(A, V, MenuList):-
read_py(A,V,Y,MenuList), % get the answer
asserta(known(Y, A, V)), % remember it
Y == yes.	% succeed or fail
"""

with open("KB.pl", "w") as text_file:
    text_file.write(KB)

In [55]:
# The code here will ask the user for input based on the askables
# It will check if the answer is known first

from pyswip.prolog import Prolog
from pyswip.easy import *
import string

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# a dictionary to store users' inputs for enabling a more natural language way of asking users
# if the users' answers exist in the dictionary, "attribute is value" will be answered as "yes"
users_answers = dict()

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y,MenuList):
    if isinstance(Y, Variable):
        # generate a menu list with index in front of the values
        menulist = [f"{idx+1}. " + str(val) for idx, val in enumerate(MenuList)]
        
        # if user has not answered the attribute
        # ask for a numeric response
        while str(A) not in users_answers:
            response = input("\nPlease choose a " + str(A) + " category: " + "(" + " / ".join(menulist) + ")" +" ---- Your choice is: ")
            if response.isnumeric():
                response = int(response)
                if response - 1 in list(range(len(menulist))):
                    users_answers[str(A)] = menulist[response-1][3:]
                else:
                    print("Choose your answer within the range of the possible answers.")
            else:
                print("Choose your numeric answer from the possible answers.")
        
        # is the user has answered the attribute with the value V and "attribute (A) is value (V)"
        # then unify the variable Y with yes, else no
        if str(A) in users_answers and users_answers[str(A)] == str(V):
            Y.unify("yes")
        else:
            Y.unify("no")
        return True
    else:
        return False
    
# define the arity of the functions
write_py.arity = 1
read_py.arity = 4

# register the foreign functions into Prolog
registerForeign(read_py)
registerForeign(write_py)

# links to the menu of each restaurant / cafe
menu = {
            "the_Visit": "https://drive.google.com/file/d/1hhxTrzMNxQZqfUoILd9tYWX5hfXX6v3Y/view",
            "maroush": "https://www.marooush.de/menus/marooush_speisekarte.pdf",
            "hacibaba": "https://www.hacibaba-berlin.de/#cat1", 
            "green_Rice": "https://drive.google.com/file/d/1Gd_arwkY0N5jhiDfMDZDsBVCQ4dl8UFr/view",
            "tangs": "https://www.tangs-kantine.de/suppen-chinesisches-restaurant-berlin-kreuzberg/",
            "parker_Bowles": "https://parker-bowles.com/wp-content/uploads/2019/08/DELI.pdf",
            "zur_Gerichtslaube": "http://www.gerichtslaube.de/wp-content/uploads/2019/11/Speisekarte-eng-19.pdf",
            "cocolo_ramen": "https://drive.google.com/file/d/1sNGL_7eRXGBHULfOvvEv1NCrAomzSMVE/view"
        } 


prolog.consult("KB.pl") # open the KB
call(retractall(known))
# query the results: the satisfied restaurant X
solution = [soln for soln in prolog.query("restaurant(X).", maxresult=1)]

# If no solution is found: the queried list is empty. Then we report no suitable results
if solution == []:
    print("\nCannot find suitable result. Try again")
# if the solution is found: solution != empty
else:
    # retrieve the name of the solution: the restaurant name
    solution = solution[0]
    # print the recommendations and link to its menu
    print("\nYou should go to " + (f"{solution['X']}" if solution else "unknown.") + " restaurant.")
    print("Here is the menu:", menu[solution['X']])


Please choose a price category: (1. low / 2. medium / 3. high) ---- Your choice is:  2


Please choose a distance category: (1. near / 2. far) ---- Your choice is:  1


Please choose a accessibility category: (1. bicycle / 2. bus) ---- Your choice is:  1


Please choose a cuisine category: (1. asian / 2. middle_eastern / 3. western) ---- Your choice is:  3


Please choose a meal_type category: (1. cafe / 2. snack / 3. restaurant) ---- Your choice is:  1


Please choose a vegan_option category: (1. vegan / 2. non_vegan) ---- Your choice is:  1


Please choose a language category: (1. english / 2. german) ---- Your choice is:  1


Please choose a opening_hours category: (1. morning / 2. afternoon / 3. evening) ---- Your choice is:  1


You should go to the_Visit restaurant.
Here is the menu: https://drive.google.com/file/d/1hhxTrzMNxQZqfUoILd9tYWX5hfXX6v3Y/view
